name: new_dataset_tests_more_epochs.ipynb

This notebook is very similar to new_dataset_tests.ipynb, but runs slightly different experiments.

Please note that this notebook is not set up with the current "experiments" folder as the destination for results. I would not recommend running this, please use `MLME-v2.0-conv_model.py` instead. 

---

https://github.com/tensorflow/tensorflow/issues/38012 for keras no module error

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/Shareddrives/NRC_Amii_Agronomics_Project/nrc-ml-plant-genomics/'

In [ ]:
import keras
import warnings, logging
import json
import numpy as np
import pandas as pd
import datetime, os

from models.base_model import BaseModel
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Input, Dense, Conv1D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping  # https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
import tensorflow as tf

from sklearn.metrics import r2_score
from scipy.stats import spearmanr # https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

warnings.filterwarnings('ignore')
logging.disable(1000)

%load_ext tensorboard

In [ ]:
pwd

'/Users/sarahdavis/Documents/internship/project_files/MPRA-DragoNN'

In [ ]:
ls

LICENSE                                 kipoi_playground.ipynb
README.md                               linear_mapping.ipynb
__init__.py                             logs/
commands_for_setup.txt                  main.py
create_final_dataset.ipynb              models/
cross_val_linear_mapping.ipynb          new_data/
data/                                   new_dataset_tests.ipynb
data_exploration.ipynb                  new_dataset_tests_default_params.ipynb
data_loader/                            new_dataset_tests_duplicate.ipynb
evaluator/                              new_dataset_tests_more_epochs.ipynb
example/                                new_models/
generate_data_format.ipynb              predict_on_NRC_data.ipynb
keras_model_loading.ipynb               requirements.txt
keras_retraining.ipynb                  requirements_exact.txt
keras_training.ipynb                    trainers/
kipoi/                                  utils/


In [ ]:
# load in model as it is from kipoi

with open('models/model.json', 'r') as json_file:
    json_savedModel = json_file.read()
    
pretrained_model = model_from_json(json_savedModel)
pretrained_model.load_weights('models/pretrained.hdf5')

In [ ]:
pretrained_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 120)          480       
_________________________________________________________________
dropout_1 (Dropout)          (None, 141, 120)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batch_normalization_2 (Batch (None, 137, 120)          480       
_________________________________________________________________
dropout_2 (Dropout)          (None, 137, 120)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 133, 120)         

In [ ]:
mapping = {"A":[1, 0, 0, 0], "T":[0, 0, 0, 1], "C":[0, 1, 0, 0], "G":[0, 0, 1, 0]}  # cross referenced with kipoi data loader

def get_ohe(sequence):  # gets sequence in format model can use
    return np.array([mapping[nt] for nt in sequence])

def get_model(treatment_dict):  # initializes model architecture
    mdl = Sequential()

    conv1_train = treatment_dict["conv1"] != 2  # bool
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(145, 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = treatment_dict["conv2"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    conv3_train = treatment_dict["conv3"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
    mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
    mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))    
    mdl.add(Dense(1, activation='linear', name="dense2"))
    
    return mdl

def set_weights(treatment_dict, mdl):  # sets appropriate model weights from pretrained
    layers_to_set = []
    if treatment_dict["conv1"] != 0: layers_to_set += [0, 1, 2]
    if treatment_dict["conv2"] != 0: layers_to_set += [3, 4, 5]
    if treatment_dict["conv3"] != 0: layers_to_set += [6, 7, 8]
    
    for i in layers_to_set:
        print(mdl.layers[i])
        pretrained_layer_weights = pretrained_model.layers[i].get_weights()
        mdl.layers[i].set_weights(pretrained_layer_weights)
    
    return mdl

# NOT USED
def return_spearman(true, pred):
    return tf.convert_to_tensor(spearmanr(true, pred)[0])

# NOT USED
def correlation(x, y):  # https://www.kaggle.com/carlolepelaars/understanding-the-metric-spearman-s-rho
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(y)
    xm, ym = x-mx, y-my
    r_num = tf.math.reduce_mean(tf.multiply(xm,ym))        
    r_den = tf.math.reduce_std(xm) * tf.math.reduce_std(ym)
    return r_num / r_den

In [ ]:
# loading in data
df = pd.read_csv("data/processed/hidra_chloroplast_15.csv")

In [ ]:
df.head()

,organelle,start_coords,end_coords,sequence,control_raw_coverage,treatment_raw_coverage,control_norm_coverage,treatment_norm_coverage,target,A,T,C,G,set
0,NC_016734.1,0,145,AATCATAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAAT...,998,316,338.31,405.52,0.26,39,40,40,26,test
1,NC_016734.1,5,150,TAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAAC...,998,318,338.31,408.08,0.27,40,40,39,26,test
2,NC_016734.1,10,155,ACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCG...,998,318,338.31,408.08,0.27,38,38,42,27,test
3,NC_016734.1,15,160,GTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGT...,998,318,338.31,408.08,0.27,40,36,42,27,test
4,NC_016734.1,20,165,GGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATT...,998,318,338.31,408.08,0.27,41,39,39,26,test


In [ ]:
# split into different datasets

train_df = df[df.set=="train"]
X_train = np.array([get_ohe(sqnc) for sqnc in train_df["sequence"]])
y_train = np.array(train_df["target"].tolist())

val_df = df[df.set=="val"]
X_val = np.array([get_ohe(sqnc) for sqnc in val_df["sequence"]])
y_val = np.array(val_df["target"].tolist())

test_df = df[df.set=="test"]
X_test = np.array([get_ohe(sqnc) for sqnc in test_df["sequence"]])
y_test = np.array(test_df["target"].tolist())

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(23458, 145, 4) (2927, 145, 4) (2927, 145, 4)
(23458,) (2927,) (2927,)


### Re-train from scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":0, 
                   "conv2":0, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)          7

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 47s 47ms/step - loss: 2.4206 - val_loss: 0.8137
Epoch 2/100
734/734 [==============================] - 35s 47ms/step - loss: 0.7633 - val_loss: 0.4903
Epoch 3/100
734/734 [==============================] - 34s 47ms/step - loss: 0.3969 - val_loss: 0.3036
Epoch 4/100
734/734 [==============================] - 34s 46ms/step - loss: 0.1636 - val_loss: 0.2691
Epoch 5/100
734/734 [==============================] - 35s 47ms/step - loss: 0.1093 - val_loss: 0.2535
Epoch 6/100
734/734 [==============================] - 35s 48ms/step - loss: 0.0639 - val_loss: 0.2209
Epoch 7/100
734/734 [==============================] - 36s 49ms/step - loss: 0.0437 - val_loss: 0.2069
Epoch 8/100
734/734 [==============================] - 33s 44ms/step - loss: 0.0332 - val_loss: 0.2053
Epoch 9/100
734/734 [==============================] - 31s 43ms/step - loss: 0.0271 - val_loss: 0.1944
Epoch 10/100
734/734 [==============================] - 31s 43ms/step - l

In [ ]:
%tensorboard --logdir logs

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9550730485166761
Val 0.21636633526044513
Test 0.11513153614742633


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.986926687846542, pvalue=0.0)
Val SpearmanrResult(correlation=0.46815244878318235, pvalue=2.3145440504455403e-159)
Test SpearmanrResult(correlation=0.3302793981577226, pvalue=1.9260677958479014e-75)


In [ ]:
ls

LICENSE                                 linear_mapping.ipynb
README.md                               logs/
__init__.py                             main.py
commands_for_setup.txt                  models/
create_final_dataset.ipynb              new_data/
cross_val_linear_mapping.ipynb          new_dataset_tests.ipynb
data/                                   new_dataset_tests_default_params.ipynb
data_exploration.ipynb                  new_dataset_tests_duplicate.ipynb
data_loader/                            new_dataset_tests_more_epochs.ipynb
evaluator/                              new_models/
example/                                new_scripts/
generate_data_format.ipynb              predict_on_NRC_data.ipynb
keras_model_loading.ipynb               requirements.txt
keras_retraining.ipynb                  requirements_exact.txt
keras_training.ipynb                    trainers/
kipoi/                                  utils/
kipoi_playground.ipynb


In [ ]:
r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0])

0.24124043437203846

In [ ]:
with open("results.csv", "w") as f:
    f.write(",R2,spearman\n")
    f.write("train,"+str(r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))+","+str(spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))+"\n")
    f.write("val,"+str(r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))+","+str(spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))+"\n")
    f.write("test,"+str(r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))+","+str(spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0])))

IndexError: invalid index to scalar variable.

### Re-train from starting point

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":1, 
                   "conv2":1, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 38s 49ms/step - loss: 0.1548 - val_loss: 0.1966
Epoch 2/100
734/734 [==============================] - 35s 48ms/step - loss: 0.0334 - val_loss: 0.1869
Epoch 3/100
734/734 [==============================] - 34s 47ms/step - loss: 0.0249 - val_loss: 0.1788
Epoch 4/100
734/734 [==============================] - 35s 47ms/step - loss: 0.0200 - val_loss: 0.1749
Epoch 5/100
734/734 [==============================] - 34s 47ms/step - loss: 0.0166 - val_loss: 0.1676
Epoch 6/100
734/734 [==============================] - 34s 47ms/step - loss: 0.0152 - val_loss: 0.1787
Epoch 7/100
734/734 [==============================] - 35s 47ms/step - loss: 0.0142 - val_loss: 0.1761
Epoch 8/100
734/734 [==============================] - 34s 47ms/step - loss: 0.0135 - val_loss: 0.1965
Epoch 9/100
734/734 [==============================] - 34s 47ms/step - loss: 0.0136 - val_loss: 0.1833
Epoch 10/100
734/734 [==============================] - 34s 47ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18270), started 0:14:48 ago. (Use '!kill 18270' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9733272884534682
Val 0.2439009604016521
Test 0.2109779255973303


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9899601158838035, pvalue=0.0)
Val SpearmanrResult(correlation=0.48257317344159556, pvalue=1.2588307389014187e-170)
Test SpearmanrResult(correlation=0.43229423636585723, pvalue=1.3642874908293733e-133)


### Train only output layer

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 11s 13ms/step - loss: 0.2091 - val_loss: 0.1659
Epoch 2/100
734/734 [==============================] - 9s 12ms/step - loss: 0.1798 - val_loss: 0.1699
Epoch 3/100
734/734 [==============================] - 8s 11ms/step - loss: 0.1704 - val_loss: 0.1758
Epoch 4/100
734/734 [==============================] - 8s 11ms/step - loss: 0.1670 - val_loss: 0.1782
Epoch 5/100
734/734 [==============================] - 8s 11ms/step - loss: 0.1583 - val_loss: 0.1797
Epoch 6/100
734/734 [==============================] - 8s 11ms/step - loss: 0.1565 - val_loss: 0.1823
Epoch 7/100
734/734 [==============================] - 9s 12ms/step - loss: 0.1541 - val_loss: 0.1863
Epoch 8/100
734/734 [==============================] - 9s 12ms/step - loss: 0.1491 - val_loss: 0.1895
Epoch 9/100
734/734 [==============================] - 9s 12ms/step - loss: 0.1491 - val_loss: 0.1912
Epoch 10/100
734/734 [==============================] - 8s 11ms/step - loss: 0.14

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18270), started 0:18:03 ago. (Use '!kill 18270' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.36138759014684463
Val 0.2518556098884309
Test 0.1538455057442818


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.6142231015764236, pvalue=0.0)
Val SpearmanrResult(correlation=0.5186443300541888, pvalue=2.75254299500723e-201)
Test SpearmanrResult(correlation=0.4242647171542155, pvalue=3.110732663352062e-128)


### Freeze Conv 1, 2, Train Conv 3 from Scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 21s 27ms/step - loss: 2.3652 - val_loss: 0.7745
Epoch 2/100
734/734 [==============================] - 17s 24ms/step - loss: 0.7774 - val_loss: 0.4580
Epoch 3/100
734/734 [==============================] - 18s 24ms/step - loss: 0.3085 - val_loss: 0.3309
Epoch 4/100
734/734 [==============================] - 17s 24ms/step - loss: 0.1528 - val_loss: 0.3099
Epoch 5/100
734/734 [==============================] - 18s 24ms/step - loss: 0.0962 - val_loss: 0.2753
Epoch 6/100
734/734 [==============================] - 18s 24ms/step - loss: 0.0662 - val_loss: 0.2462
Epoch 7/100
734/734 [==============================] - 19s 26ms/step - loss: 0.0437 - val_loss: 0.2260
Epoch 8/100
734/734 [==============================] - 20s 27ms/step - loss: 0.0368 - val_loss: 0.2253
Epoch 9/100
734/734 [==============================] - 19s 26ms/step - loss: 0.0273 - val_loss: 0.2228
Epoch 10/100
734/734 [==============================] - 19s 26ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18270), started 0:34:55 ago. (Use '!kill 18270' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9808997557103227
Val 0.10790987926415563
Test 0.057492984290018145


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9929224186001404, pvalue=0.0)
Val SpearmanrResult(correlation=0.3145536060830988, pvalue=3.1210976408112577e-68)
Test SpearmanrResult(correlation=0.22313449752737607, pvalue=2.395036362764694e-34)


### Freeze Conv 1, 2, Train Conv 3 from Init

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 22s 27ms/step - loss: 0.1870 - val_loss: 0.2304
Epoch 2/100
734/734 [==============================] - 19s 25ms/step - loss: 0.0538 - val_loss: 0.2065
Epoch 3/100
734/734 [==============================] - 19s 26ms/step - loss: 0.0340 - val_loss: 0.1899
Epoch 4/100
734/734 [==============================] - 18s 25ms/step - loss: 0.0283 - val_loss: 0.1856
Epoch 5/100
734/734 [==============================] - 18s 24ms/step - loss: 0.0242 - val_loss: 0.1991
Epoch 6/100
734/734 [==============================] - 18s 24ms/step - loss: 0.0223 - val_loss: 0.2024
Epoch 7/100
734/734 [==============================] - 18s 25ms/step - loss: 0.0221 - val_loss: 0.1935
Epoch 8/100
734/734 [==============================] - 20s 28ms/step - loss: 0.0203 - val_loss: 0.1833
Epoch 9/100
734/734 [==============================] - 19s 25ms/step - loss: 0.0193 - val_loss: 0.1905
Epoch 10/100
734/734 [==============================] - 17s 24ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18270), started 0:46:52 ago. (Use '!kill 18270' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9629402427068685
Val 0.19670003919546697
Test 0.09247332760955385


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.992360968151913, pvalue=0.0)
Val SpearmanrResult(correlation=0.43533615454640867, pvalue=1.1665585659417078e-135)
Test SpearmanrResult(correlation=0.3156834718041059, pvalue=9.786192991418566e-69)


---
### Spearman Results

0 for init random weights & train  
1 for load pre-trained weights and train  
2 for load pre-trained weights and freeze  

First go:
000: val = 0.47753801170363896, test = 0.2866754020651326  
111: val = 0.47855750422978954, test = 0.279400514316224  
222: val = 0.5858565342434344, test = 0.3741217612323338  
220: val = 0.41782505305420903, test = 0.23175390117095385  
221: val = 0.4622724735247824, test = 0.26885707718958146  
2220: val = 0.5984249204802108, test = 0.32715495828441904

Second go:
000: val = 0.44123205768583007, test = 0.17504952705223212  
111: val = 0.4053507361644881, test = 0.2592776174804178  
222: val = 0.5290568130486669, test = 0.3636127380447914  
220: val = 0.392220547475502, test = 0.24043305401824872 
221: val = 0.40291026906804733, test = 0.22825701637010898
2220: val = 0.5988624237048076, test = 0.3301200533685432

### Train only output layer

In [ ]:
def get_model(treatment_dict):  # initializes model architecture
    mdl = Sequential()

    conv1_train = treatment_dict["conv1"] != 2  # bool
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(145, 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = treatment_dict["conv2"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    conv3_train = treatment_dict["conv3"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
    mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
    mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))    
    mdl.add(Dense(12, activation='linear', name="dense2", trainable=False))
    mdl.add(Dense(1, activation='linear', name="dense3"))
    
    return mdl

def set_weights(treatment_dict, mdl):  # sets appropriate model weights from pretrained
    layers_to_set = [i for i in range(11)]
    
    for i in layers_to_set:
        print(mdl.layers[i])
        pretrained_layer_weights = pretrained_model.layers[i].get_weights()
        mdl.layers[i].set_weights(pretrained_layer_weights)
    
    return mdl

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
734/734 [==============================] - 19s 23ms/step - loss: 0.4410 - val_loss: 0.1940
Epoch 2/100
734/734 [==============================] - 13s 18ms/step - loss: 0.2251 - val_loss: 0.1713
Epoch 3/100
734/734 [==============================] - 12s 16ms/step - loss: 0.2093 - val_loss: 0.1674
Epoch 4/100
734/734 [==============================] - 11s 15ms/step - loss: 0.2093 - val_loss: 0.1665
Epoch 5/100
734/734 [==============================] - 11s 15ms/step - loss: 0.2040 - val_loss: 0.1690
Epoch 6/100
734/734 [==============================] - 11s 15ms/step - loss: 0.2064 - val_loss: 0.1690
Epoch 7/100
734/734 [==============================] - 14s 19ms/step - loss: 0.2064 - val_loss: 0.1668
Epoch 8/100
734/734 [==============================] - 16s 22ms/step - loss: 0.2058 - val_loss: 0.1677
Epoch 9/100
734/734 [==============================] - 16s 22ms/step - loss: 0.2053 - val_loss: 0.1658
Epoch 10/100
734/734 [==============================] - 13s 17ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 18270), started 0:59:21 ago. (Use '!kill 18270' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.2618717956334292
Val 0.26729943961851377
Test 0.1886281739966693


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.5198213241496843, pvalue=0.0)
Val SpearmanrResult(correlation=0.51172420709206, pvalue=4.081904428015674e-195)
Test SpearmanrResult(correlation=0.44716308158447066, pvalue=6.649003706919414e-144)


---
# 40

In [ ]:
# loading in data
df = pd.read_csv("new_data/hidra_chloroplast_40.csv")

In [ ]:
df.head()

,organelle,start_coords,end_coords,sequence,control_raw_coverage,treatment_raw_coverage,control_norm_coverage,treatment_norm_coverage,target,A,T,C,G,set
0,NC_016734.1,0,145,AATCATAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAAT...,998,316,338.31,405.52,0.26,39,40,40,26,test
1,NC_016734.1,5,150,TAATAACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAAC...,998,318,338.31,408.08,0.27,40,40,39,26,test
2,NC_016734.1,10,155,ACTTGGTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCG...,998,318,338.31,408.08,0.27,38,38,42,27,test
3,NC_016734.1,15,160,GTCCCGGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGT...,998,318,338.31,408.08,0.27,40,36,42,27,test
4,NC_016734.1,20,165,GGGCATCACGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATT...,998,318,338.31,408.08,0.27,41,39,39,26,test


In [ ]:
# split into different datasets

train_df = df[df.set=="train"]
X_train = np.array([get_ohe(sqnc) for sqnc in train_df["sequence"]])
y_train = np.array(train_df["target"].tolist())

val_df = df[df.set=="val"]
X_val = np.array([get_ohe(sqnc) for sqnc in val_df["sequence"]])
y_val = np.array(val_df["target"].tolist())

test_df = df[df.set=="test"]
X_test = np.array([get_ohe(sqnc) for sqnc in test_df["sequence"]])
y_test = np.array(test_df["target"].tolist())

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(21844, 145, 4) (2684, 145, 4) (2684, 145, 4)
(21844,) (2684,) (2684,)


### Re-train from scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":0, 
                   "conv2":0, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 44s 62ms/step - loss: 0.1570 - val_loss: 0.1961
Epoch 2/100
683/683 [==============================] - 39s 58ms/step - loss: 0.0396 - val_loss: 0.1793
Epoch 3/100
683/683 [==============================] - 39s 57ms/step - loss: 0.0280 - val_loss: 0.1810
Epoch 4/100
683/683 [==============================] - 39s 57ms/step - loss: 0.0219 - val_loss: 0.1782
Epoch 5/100
683/683 [==============================] - 38s 56ms/step - loss: 0.0189 - val_loss: 0.1916
Epoch 6/100
683/683 [==============================] - 38s 56ms/step - loss: 0.0172 - val_loss: 0.1808
Epoch 7/100
683/683 [==============================] - 39s 57ms/step - loss: 0.0152 - val_loss: 0.1817
Epoch 8/100
683/683 [==============================] - 45s 66ms/step - loss: 0.0137 - val_loss: 0.1829
Epoch 9/100
683/683 [==============================] - 43s 64ms/step - loss: 0.0134 - val_loss: 0.1836
Epoch 10/100
683/683 [==============================] - 43s 63ms/step - l

In [ ]:
%tensorboard --logdir logs

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9672868646098323
Val 0.37122746384963246
Test 0.21161039778814295


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9860352600373611, pvalue=0.0)
Val SpearmanrResult(correlation=0.615802985507954, pvalue=5.416410286351154e-280)
Test SpearmanrResult(correlation=0.4683716368913346, pvalue=1.9234648943495221e-146)


### Re-train from starting point

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":1, 
                   "conv2":1, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 48s 66ms/step - loss: 0.1690 - val_loss: 0.1892
Epoch 2/100
683/683 [==============================] - 43s 63ms/step - loss: 0.0465 - val_loss: 0.1919
Epoch 3/100
683/683 [==============================] - 43s 63ms/step - loss: 0.0299 - val_loss: 0.1861
Epoch 4/100
683/683 [==============================] - 45s 67ms/step - loss: 0.0245 - val_loss: 0.1798
Epoch 5/100
683/683 [==============================] - 44s 64ms/step - loss: 0.0194 - val_loss: 0.1847
Epoch 6/100
683/683 [==============================] - 44s 64ms/step - loss: 0.0183 - val_loss: 0.1821
Epoch 7/100
683/683 [==============================] - 45s 66ms/step - loss: 0.0149 - val_loss: 0.1766
Epoch 8/100
683/683 [==============================] - 45s 65ms/step - loss: 0.0143 - val_loss: 0.1820
Epoch 9/100
683/683 [==============================] - 46s 67ms/step - loss: 0.0133 - val_loss: 0.1847
Epoch 10/100
683/683 [==============================] - 48s 71ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30228), started 0:18:38 ago. (Use '!kill 30228' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9757126934753173
Val 0.37685782764857967
Test 0.20896743975473753


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9908803104883138, pvalue=0.0)
Val SpearmanrResult(correlation=0.6222976264542276, pvalue=1.3684450046212899e-287)
Test SpearmanrResult(correlation=0.4372889250980363, pvalue=8.823394655917018e-126)


### Train only output layer

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)         

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 10s 13ms/step - loss: 0.2295 - val_loss: 0.2055
Epoch 2/100
683/683 [==============================] - 9s 13ms/step - loss: 0.2080 - val_loss: 0.2007
Epoch 3/100
683/683 [==============================] - 9s 13ms/step - loss: 0.2009 - val_loss: 0.1989
Epoch 4/100
683/683 [==============================] - 9s 13ms/step - loss: 0.2072 - val_loss: 0.1987
Epoch 5/100
683/683 [==============================] - 8s 12ms/step - loss: 0.2001 - val_loss: 0.1987
Epoch 6/100
683/683 [==============================] - 8s 12ms/step - loss: 0.2020 - val_loss: 0.1963
Epoch 7/100
683/683 [==============================] - 9s 13ms/step - loss: 0.2034 - val_loss: 0.1963
Epoch 8/100
683/683 [==============================] - 8s 12ms/step - loss: 0.2018 - val_loss: 0.1958
Epoch 9/100
683/683 [==============================] - 8s 12ms/step - loss: 0.2036 - val_loss: 0.1958
Epoch 10/100
683/683 [==============================] - 8s 12ms/step - loss: 0.20

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30228), started 0:27:58 ago. (Use '!kill 30228' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.24092483509832452
Val 0.31761735410061576
Test 0.2555477413486553


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.49743165674928225, pvalue=0.0)
Val SpearmanrResult(correlation=0.5505672205007068, pvalue=1.3116075375682348e-212)
Test SpearmanrResult(correlation=0.5506971933641037, pvalue=9.955655059416527e-213)


### Freeze Conv 1, 2, Train Conv 3 from Scratch

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":0}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)        

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 20s 28ms/step - loss: 0.2087 - val_loss: 0.2217
Epoch 2/100
683/683 [==============================] - 18s 26ms/step - loss: 0.0916 - val_loss: 0.2020
Epoch 3/100
683/683 [==============================] - 17s 25ms/step - loss: 0.0551 - val_loss: 0.2084
Epoch 4/100
683/683 [==============================] - 17s 24ms/step - loss: 0.0413 - val_loss: 0.2001
Epoch 5/100
683/683 [==============================] - 17s 25ms/step - loss: 0.0348 - val_loss: 0.1982
Epoch 6/100
683/683 [==============================] - 17s 25ms/step - loss: 0.0316 - val_loss: 0.1960
Epoch 7/100
683/683 [==============================] - 18s 26ms/step - loss: 0.0279 - val_loss: 0.1899
Epoch 8/100
683/683 [==============================] - 17s 25ms/step - loss: 0.0262 - val_loss: 0.1933
Epoch 9/100
683/683 [==============================] - 17s 25ms/step - loss: 0.0241 - val_loss: 0.1924
Epoch 10/100
683/683 [==============================] - 16s 24ms/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30228), started 0:40:08 ago. (Use '!kill 30228' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9737901624385901
Val 0.34615371424875574
Test 0.18750562898925638


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.9912703598654496, pvalue=0.0)
Val SpearmanrResult(correlation=0.5988764861697347, pvalue=5.269946749173147e-261)
Test SpearmanrResult(correlation=0.4366262612574109, pvalue=2.3075557715239193e-125)


### Freeze Conv 1, 2, Train Conv 3 from Init

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":1}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)        

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 25s 33ms/step - loss: 0.1915 - val_loss: 0.2035
Epoch 2/100
683/683 [==============================] - 22s 32ms/step - loss: 0.0815 - val_loss: 0.2139
Epoch 3/100
683/683 [==============================] - 23s 33ms/step - loss: 0.0531 - val_loss: 0.2126
Epoch 4/100
683/683 [==============================] - 22s 33ms/step - loss: 0.0414 - val_loss: 0.2232
Epoch 5/100
683/683 [==============================] - 22s 32ms/step - loss: 0.0353 - val_loss: 0.2137
Epoch 6/100
683/683 [==============================] - 20s 30ms/step - loss: 0.0315 - val_loss: 0.2057
Epoch 7/100
683/683 [==============================] - 1825s 3s/step - loss: 0.0280 - val_loss: 0.2152
Epoch 8/100
683/683 [==============================] - 4669s 7s/step - loss: 0.0268 - val_loss: 0.2145
Epoch 9/100
683/683 [==============================] - 3652s 5s/step - loss: 0.0243 - val_loss: 0.2132
Epoch 10/100
683/683 [==============================] - 3673s 5s/step - l

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30228), started 11:55:00 ago. (Use '!kill 30228' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.9794030053637053
Val 0.28301586737718565
Test 0.16496629836058974


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.991383239088942, pvalue=0.0)
Val SpearmanrResult(correlation=0.5281418799925951, pvalue=1.0212364752360976e-192)
Test SpearmanrResult(correlation=0.3888156681004329, pvalue=1.3815857403885453e-97)


---
### Spearman Results

0 for init random weights & train  
1 for load pre-trained weights and train  
2 for load pre-trained weights and freeze  

First go:
000: val = 0.47753801170363896, test = 0.2866754020651326  
111: val = 0.47855750422978954, test = 0.279400514316224  
222: val = 0.5858565342434344, test = 0.3741217612323338  
220: val = 0.41782505305420903, test = 0.23175390117095385  
221: val = 0.4622724735247824, test = 0.26885707718958146  
2220: val = 0.5984249204802108, test = 0.32715495828441904

Second go:
000: val = 0.44123205768583007, test = 0.17504952705223212  
111: val = 0.4053507361644881, test = 0.2592776174804178  
222: val = 0.5290568130486669, test = 0.3636127380447914  
220: val = 0.392220547475502, test = 0.24043305401824872 
221: val = 0.40291026906804733, test = 0.22825701637010898
2220: val = 0.5988624237048076, test = 0.3301200533685432

### Train only output layer

In [ ]:
def get_model(treatment_dict):  # initializes model architecture
    mdl = Sequential()

    conv1_train = treatment_dict["conv1"] != 2  # bool
    mdl.add(Conv1D(120, 5, activation='relu', input_shape=(145, 4), name="1DConv_1", trainable=conv1_train))
    mdl.add(BatchNormalization(name="batchNorm1", trainable=conv1_train))
    mdl.add(Dropout(0.1, name="drop1"))

    conv2_train = treatment_dict["conv2"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_2", trainable=conv2_train))
    mdl.add(BatchNormalization(name="batchNorm2", trainable=conv2_train))
    mdl.add(Dropout(0.1, name="drop2"))

    conv3_train = treatment_dict["conv3"] != 2  # bool 
    mdl.add(Conv1D(120, 5, activation='relu', name="1DConv_3", trainable=conv3_train))
    mdl.add(BatchNormalization(name="batchNorm3", trainable=conv3_train))
    mdl.add(Dropout(0.1, name="drop3"))

    mdl.add(Flatten(name="flat"))    
    mdl.add(Dense(12, activation='linear', name="dense2", trainable=False))
    mdl.add(Dense(1, activation='linear', name="dense3"))
    
    return mdl

def set_weights(treatment_dict, mdl):  # sets appropriate model weights from pretrained
    layers_to_set = [i for i in range(11)]
    
    for i in layers_to_set:
        print(mdl.layers[i])
        pretrained_layer_weights = pretrained_model.layers[i].get_weights()
        mdl.layers[i].set_weights(pretrained_layer_weights)
    
    return mdl

In [ ]:
# 0 for init random weights & train
# 1 for load pre-trained weights and train
# 2 for load pre-trained weights and freeze

layer_treatment = {"conv1":2, 
                   "conv2":2, 
                   "conv3":2}

model = get_model(layer_treatment)
model = set_weights(layer_treatment, model)

In [ ]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1DConv_1 (Conv1D)            (None, 141, 120)          2520      
_________________________________________________________________
batchNorm1 (BatchNormalizati (None, 141, 120)          480       
_________________________________________________________________
drop1 (Dropout)              (None, 141, 120)          0         
_________________________________________________________________
1DConv_2 (Conv1D)            (None, 137, 120)          72120     
_________________________________________________________________
batchNorm2 (BatchNormalizati (None, 137, 120)          480       
_________________________________________________________________
drop2 (Dropout)              (None, 137, 120)          0         
_________________________________________________________________
1DConv_3 (Conv1D)            (None, 133, 120)        

In [ ]:
rm -rf ./logs/

In [ ]:
model.compile(optimizer='adam', 
              loss='mean_squared_error') 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)  # https://stackoverflow.com/questions/59894720/keras-and-tensorboard-attributeerror-sequential-object-has-no-attribute-g

es_callback = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc_callback = ModelCheckpoint('new_models/best_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, es_callback, mc_callback])

Epoch 1/100
683/683 [==============================] - 25s 33ms/step - loss: 0.3967 - val_loss: 0.2392
Epoch 2/100
683/683 [==============================] - 3612s 5s/step - loss: 0.2252 - val_loss: 0.2163
Epoch 3/100
683/683 [==============================] - 70s 103ms/step - loss: 0.2105 - val_loss: 0.2069
Epoch 4/100
683/683 [==============================] - 17s 25ms/step - loss: 0.2057 - val_loss: 0.2013
Epoch 5/100
683/683 [==============================] - 16s 24ms/step - loss: 0.2041 - val_loss: 0.1995
Epoch 6/100
683/683 [==============================] - 19s 28ms/step - loss: 0.2031 - val_loss: 0.1980
Epoch 7/100
683/683 [==============================] - 20s 29ms/step - loss: 0.2019 - val_loss: 0.1973
Epoch 8/100
683/683 [==============================] - 17s 25ms/step - loss: 0.2008 - val_loss: 0.1965
Epoch 9/100
683/683 [==============================] - 18s 26ms/step - loss: 0.2034 - val_loss: 0.1953
Epoch 10/100
683/683 [==============================] - 19s 29ms/step - 

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 30228), started 14:14:56 ago. (Use '!kill 30228' to kill it.)

In [ ]:
saved_model = load_model('new_models/best_model.h5')

In [ ]:
# R^2
print("Train", r2_score(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", r2_score(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", r2_score(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train 0.24124043437203846
Val 0.3174910057945999
Test 0.25752724468248067


In [ ]:
# Spearman
print("Train", spearmanr(y_train, saved_model.predict(X_train).reshape(1, -1)[0]))
print("Val", spearmanr(y_val, saved_model.predict(X_val).reshape(1, -1)[0]))
print("Test", spearmanr(y_test, saved_model.predict(X_test).reshape(1, -1)[0]))

Train SpearmanrResult(correlation=0.49736508184825123, pvalue=0.0)
Val SpearmanrResult(correlation=0.5506824122889169, pvalue=1.0272808113293604e-212)
Test SpearmanrResult(correlation=0.5514594370946222, pvalue=1.9715563393393002e-213)
